In [14]:
import pandas as pd
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

In [ ]:
try:
    FILES
    FILE = FILES[2]
    FOLDER
except NameError:
    FILE = ['03_sell_in_lag.parquet']
    FOLDER = 'DefaultFolder'

    
print(f"------------------------------COMIENZA-----------------------------\n[{FOLDER}/{FILE}")

------------------------------COMIENZA-----------------------------
[DefaultFolder/['01_sell_in_group.parquet', '01_prod_stats.parquet', '01_prod_data.parquet', '01_stock_data.parquet']


In [15]:
# df = pd.read_parquet("sell_in_norm.parquet")
# prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')
# prod_data = pd.read_parquet('prod_data.parquet')

df = pd.read_parquet(f"{FOLDER}/{FILES[1][0]}")
prod_data = pd.read_parquet(f"{FOLDER}/{FILES[0][2]}")
prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')

## Pendientes
---
### operaciones sobre tn
* HECHO - razones con lags
* HECHO - marcar cero real
* HECHO - date features:
    - HECHO - mes
    - HECHO - quarter
    - HECHO - mes en el quarter
* HECHO - tiempos de monotonia de la funcion
* maximo salto
* expanding mean

### basadas en producto
* HECHO - info del producto
* HECHO porcentual de venta
    - HECHO del mes
    - HECHO del quarter
    - HECHO del año
* porcentual de categoria
* HECHO - stats de categoria1 y 2
* HECHO - stats de brand

## Generales

In [16]:
# date features
df['periodo_dt'] = pd.to_datetime(df['periodo'], format='%Y%m')
df = df.sort_values(by=['product_id', 'periodo_dt'])
df['mes'] = df['periodo_dt'].dt.month
df['quarter'] = df['periodo_dt'].dt.quarter
df['month_in_quarter'] = (df['periodo_dt'].dt.month - 1) % 3 + 1
df['year'] = df['periodo_dt'].dt.year
df['yearquarter'] = df['periodo_dt'].dt.to_period('Q').astype('str').astype('category')

print(df[df['periodo'] == 201912].shape)

(780, 20)


## Operaciones sobre tn

In [17]:
# marcar ceros
df['tn_cero'] = df['tn'] < df['median_tn'] / 10

In [18]:
lags = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15,18,21,24,30,36]
lags

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 18, 21, 24, 30, 36]

In [19]:
# lag features
for lag in lags:
    # rolling max
    df[f'max_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).max() == df['tn_norm']
    # rolling min
    df[f'min_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).min() == df['tn_norm']
    # rolling avg
    df[f'avg_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).mean()
    # lags
    df[f'tn_lag_{lag}'] = df.groupby('product_id')['tn_norm'].shift(lag)
    # primer derivada / delta
    df[f'tn_diff_{lag}'] = df['tn_norm'].diff(lag)
    # ratios con lags
    df[f'ratio_{lag}'] = (-df[f'tn_diff_{lag}'])/df[f'tn_lag_{lag}']

for deriv in range(1, 13):
    # segunda derivada
    df[f'tn_diff2_{deriv}'] = df[f'tn_diff_{deriv}'].diff()

In [20]:
# monotonia

for i in lags:
    df[f'crece_{i+1}'] = (df.groupby('product_id')['tn_norm'].shift(i) - df.groupby('product_id')['tn_norm'].shift(i+1)) > 0

crece_columns = df.filter(like='crece_')

df['crece_sum'] = crece_columns.sum(axis=1)
df['decrece_sum'] = 12 - crece_columns.sum(axis=1) #corregir para los primeros

## nivel producto

In [21]:
df = pd.merge(df, prod_data, on='product_id', how='left', suffixes=('', ''))

In [22]:
print(df[df['periodo'] == 201912].shape)

(780, 166)


In [23]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [24]:
variables = ['cat1', 'cat2', 'brand']
var_stats = []

for variable in variables:
    var_stat = df.groupby(variable).apply(lambda group: pd.Series({
        f'{variable}_total': group['tn'].sum(),
        #f'{variable}_min': group['tn'].min(),
        #f'{variable}_average': group['tn'].mean(),
        f'{variable}_median': group['tn'].median(),
        #f'{variable}_std_dev': group['tn'].std(),
        #f'{variable}_iqr': (group['tn'].quantile(0.75) - group['tn'].quantile(0.25)),
        f'{variable}_max': group['tn'].max()
    }))
    var_stat.reset_index(inplace=True)
    var_stats.append(var_stat)

# Merge yearly sales with the products dataframe
#df_prod = df_prod.merge(df_temp, on='product_id', how='left')

In [25]:
for i in range(len(variables)):
    var_stat = var_stats[i]
    df = pd.merge(df, var_stat, on=variables[i], how='left')

In [26]:
tiempos = ['periodo','yearquarter','year']

for tiempo in tiempos:
    tiempo_sums = df.groupby(tiempo)['tn'].sum().reset_index()
    tiempo_sums.rename(columns={'tn': f'tn_total_{tiempo}'}, inplace=True)
    df = df.merge(tiempo_sums, on=tiempo)
    df[f'prop_product_{tiempo}'] = (df['tn'] / df[f'tn_total_{tiempo}']) * 100
    df.drop(columns=f'tn_total_{tiempo}', inplace=True)

In [27]:
variables = ['cat1','cat2','brand']

for tiempo in tiempos:
    for variable in variables:
        combi_sums = df.groupby([tiempo, variable])['tn'].sum().reset_index()
        combi_sums = df.groupby([tiempo, variable])['tn'].sum().reset_index()
        combi_sums.rename(columns={'tn': f'tn_total_{tiempo}_{variable}'}, inplace=True)
        df = df.merge(combi_sums, on=[tiempo, variable])
        df[f'prop_product_{tiempo}_{variable}'] = (df['tn'] / df[f'tn_total_{tiempo}_{variable}']) * 100
        df.drop(columns=f'tn_total_{tiempo}_{variable}', inplace=True)

## ajustes finales pre export

In [28]:
df = df.sort_values(by=['product_id', 'periodo'])

In [29]:
df.head(15)

,product_id,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,median_tn,...,prop_product_year,prop_product_periodo_cat1,prop_product_periodo_cat2,prop_product_periodo_brand,prop_product_yearquarter_cat1,prop_product_yearquarter_cat2,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand
0,20001,201701,934.77222,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.233301,5.826401,11.654367,80.066430,1.690335,3.436027,24.486934,0.375661,0.729880,5.127212
4,20001,201702,798.01620,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.199169,4.981156,10.162910,77.736070,1.443041,2.933340,20.904526,0.320702,0.623100,4.377108
8,20001,201703,1303.35771,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.325292,5.609080,11.501565,80.287383,2.356843,4.790870,34.142258,0.523786,1.017676,7.148899
12,20001,201704,1069.96130,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.267041,5.473412,11.826828,79.904957,1.615894,3.192987,21.535487,0.429990,0.835437,5.868723
16,20001,201705,1502.20132,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.374920,6.395256,12.299625,82.754738,2.268679,4.482880,30.235334,0.603696,1.172935,8.239553
20,20001,201706,1520.06539,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.379378,6.558464,12.409262,83.792819,2.295658,4.536191,30.594891,0.610875,1.186884,8.337537
24,20001,201707,1030.67391,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.257236,5.373548,10.525361,81.058064,1.621715,3.094388,23.533215,0.414201,0.804761,5.653232
28,20001,201708,1267.39462,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.316316,6.253265,12.725968,84.132703,1.994184,3.805094,28.938221,0.509333,0.989595,6.951642
32,20001,201709,1316.94604,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.328684,5.463075,9.714550,82.221381,2.072151,3.953862,30.069621,0.529247,1.028286,7.223430
36,20001,201710,1439.75563,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.359334,5.968643,10.959619,80.672567,2.257953,4.228433,28.419212,0.578601,1.124177,7.897039


In [30]:
#df.to_parquet('sell_in_lag.parquet', index=False)
df.to_parquet(f'{FOLDER}/{FILE[0]}', index=False)

In [ ]:
#estado_control = f"03_features Terminado - {nombrefile} - {datetime.now()}"

Notebook 1 terminado


In [ ]:
print(f" {FOLDER}/{FILE}\n------------------------------FINALIZA-----------------------------")

 DefaultFolder/['01_sell_in_group.parquet', '01_prod_stats.parquet', '01_prod_data.parquet', '01_stock_data.parquet']
------------------------------FINALIZA-----------------------------
